## В классе
Придумать спецификацию класса агента – игрока

In [ ]:
class AgentTicTac:
    def __init__(is_zero):
    #is_zero == True если нолик
        self.is_zero = True

    def get_action(state):
    # пример state = [[1,0,1],[0,1,0],[0,1,0]] 1 - это крестик, 0 - это нолик, -1 - пусто
        return (i,j) # возвращаем координаты хода

    def is_done(state, reward)
    # reward - вознаграждение 1 если выиграли , вызывается когда игра закончена
        

## Дома
Написать класс, который берет два экземпляра классов агентов и играет

Реализовать игроков на на основе обучения с подкреплением и эволюционного подхода

Поведём состязание

Проверить: как лучше – корректировать ценности после всех ходов или только "жадных"?

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

In [18]:
#проверка победы
def check_win(state, label):
    for i in range(3):
        if np.all(state[i,:] == int(label)):
            return True
        elif np.all(state[:,i] == int(label)):
            return True
    if np.all(np.fliplr(state).diagonal() == int(label)):
        return True
    if np.all(state.diagonal() == int(label)):
        return True
    return False

#вывод поля состояния в виде матрицы
def get_state_from_code(code):
    return np.array(re.findall(r'\d', code)).reshape((3,3)).astype('int')-1

#
def get_code_from_state(state):
    result = ''
    for s in (state.flatten()+1).astype('str'):
        result += s
    return result

#гениратор состояний
def state_generate(label):
    states = []
    values = []
    for i in range(19683):
        s = np.base_repr(i,base=3)
        s = '0'*(9-len(s)) + s
        states.append(s)
        if check_win(get_state_from_code(s), label):
            values.append(1)
        else:
            values.append(0.5)
    states = np.array(states)
    return pd.DataFrame({'states':states, 'value':values}).set_index('states')
        

## функция выбора след.хода

скопировала твой гениратор и добавила своего кода, поправь пожалст, а то прям видно что писал недопрограммист))

In [51]:
#получаем индексы свободных полей
def get_free_field(state):
    index = []
    for i in range(3):
        for j in range(3):
            if state[i][j]==-1:
                index.append((i,j))
    return index

In [52]:
#получение списка следующих возможных ходов
def get_possible_state(state, label):
    pos_state = []
    index = get_free_field(state)
    for i,j in index:
        s=state
        s[i][j]=label
        pos_state.append(get_code_from_state(s))
    return pos_state

In [60]:
#генирация состояний со следующими ходами
#гениратор состояний
def state_generate(label):
    states = []
    values = []
    for i in range(19683):
        s = np.base_repr(i,base=3)
        s = '0'*(9-len(s)) + s
        states.append(s)
        if check_win(get_state_from_code(s), label):
            values.append(1)
        else:
            values.append(0.5)
    states = np.array(states)
    
    #мой мозг не смог лучше придумать, оптимизируй это, если захочешь
    pos_st = []
    label = True
    for state_code in states:
        pos_st.append(get_possible_state(get_state_from_code(state_code),int(label)))
    
    return pd.DataFrame({'states':states, 'value':values,'possible_states':pos_st}).set_index('states')
        


In [46]:
get_state_from_code('222211010')

array([[ 1,  1,  1],
       [ 1,  0,  0],
       [-1,  0, -1]])

In [63]:
get_code_from_state(np.array([[ 1,  1,  1],[ 1,  0,  0], [-1, -1, -1]]))

'222211000'

In [61]:
st = state_generate(True)

In [62]:
st

,value,possible_states
states,,
000000000,0.5,"[100000000, 110000000, 111000000, 111100000, 1..."
000000001,0.5,"[100000001, 110000001, 111000001, 111100001, 1..."
000000002,0.5,"[100000002, 110000002, 111000002, 111100002, 1..."
000000010,0.5,"[100000010, 110000010, 111000010, 111100010, 1..."
000000011,0.5,"[100000011, 110000011, 111000011, 111100011, 1..."
000000012,0.5,"[100000012, 110000012, 111000012, 111100012, 1..."
000000020,0.5,"[100000020, 110000020, 111000020, 111100020, 1..."
000000021,0.5,"[100000021, 110000021, 111000021, 111100021, 1..."
000000022,0.5,"[100000022, 110000022, 111000022, 111100022, 1..."
